In [1]:
import httpx

In [15]:
TAGS_CNT = 999
PAGE_CNT = 4
res = []
for i in range(1, PAGE_CNT+1):
    data = httpx.get(f"https://taginfo.openstreetmap.org/api/4/tags/popular?sortname=count_all&sortorder=desc&rp={TAGS_CNT}&page={i}").json()
    res += data['data']

len(res)

3996

In [16]:
import pandas as pd

df = pd.DataFrame(res)
df.head()

,key,value,in_wiki,count_all,count_all_fraction,count_nodes,count_nodes_fraction,count_ways,count_ways_fraction,count_relations,count_relations_fraction,projects
0,building,yes,1,487959537,0.0476,296035,0.0013,487018628,0.4731,644874,0.0527,13
1,highway,residential,1,66438369,0.0065,37,0.0000,66438305,0.0645,27,0.0000,37
2,building,house,1,56044929,0.0055,229935,0.0010,55715434,0.0541,99560,0.0081,11
3,highway,service,1,56044254,0.0055,56,0.0000,56040456,0.0544,3742,0.0003,33
4,surface,asphalt,1,27513806,0.0027,96384,0.0004,27392917,0.0266,24505,0.0020,26


In [17]:
df_with_wiki = df[df['in_wiki']==1]

In [32]:
len(df_with_wiki)

1066

In [33]:
def get_wiki(key, value):
    try:
        res = httpx.get(
            "https://taginfo.openstreetmap.org/api/4/tag/wiki_pages",
            params = {'key': key, 'value': value},
            timeout=15.0
        )
        for r in res.json()['data']:
            if r['lang']=='en':return r
    except TimeoutError:
        return 'timeout'

In [34]:
res = get_wiki('mountain_pass', 'yes')
print(res)

{'lang': 'en', 'dir': 'ltr', 'language': 'English', 'language_en': 'English', 'title': 'Tag:mountain_pass=yes', 'description': 'Marks the highest point of a mountain road, railway or path as it passes over a crest.', 'image': {'title': 'File:Mountain pass.jpg', 'width': 300, 'height': 225, 'mime': 'image/jpeg', 'image_url': 'https://wiki.openstreetmap.org/w/images/e/e3/Mountain_pass.jpg', 'thumb_url_prefix': 'https://wiki.openstreetmap.org/w/images/thumb/e/e3/Mountain_pass.jpg/', 'thumb_url_suffix': 'px-Mountain_pass.jpg'}, 'on_node': True, 'on_way': False, 'on_area': False, 'on_relation': False, 'tags_implies': [], 'tags_combination': ['ele', 'name'], 'tags_linked': ['ele', 'mountain_pass', 'mountain_pass=yes', 'name', 'natural=saddle'], 'status': 'approved'}


In [35]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [37]:
import asyncio
df_with_wiki['wiki'] = df_with_wiki.parallel_apply(
    lambda rw:get_wiki(rw['key'], rw['value']), axis=1
)

/var/folders/z5/ks9fxjt91y1dgv38xzprh08m0000gn/T/ipykernel_32039/263575737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_wiki['wiki'] = df_with_wiki.parallel_apply(


In [50]:
df_with_wiki = df_with_wiki[df_with_wiki['wiki'].notnull()]

In [45]:
df_with_wiki.to_pickle("tags.pkl")

In [52]:
df_with_wiki['description'] = df_with_wiki['wiki'].apply(
    lambda x: x['description']
)

/var/folders/z5/ks9fxjt91y1dgv38xzprh08m0000gn/T/ipykernel_32039/2528192062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_wiki['description'] = df_with_wiki['wiki'].apply(


In [54]:
df_with_wiki[['key', 'value', 'count_all_fraction', 'description']].to_json(
    "tags.json", orient="records"
)

In [55]:
df['count_all_fraction'].sum()

0.1952

## Keys

In [62]:
from tqdm import tqdm
TAGS_CNT = 999
PAGE_CNT = 5
keys_res = []
for i in tqdm(range(1, PAGE_CNT+1)):
    data = httpx.get(f"https://taginfo.openstreetmap.org/api/4/keys/all?sortname=count_all&sortorder=desc&rp={TAGS_CNT}&page={i}").json()
    keys_res += data['data']

len(keys_res)

100%|██████████████████████████████████████| 5/5 [00:08<00:00,  1.67s/it]


4995

In [72]:
df_keys = pd.DataFrame(keys_res)
df_keys['count_all_fraction'].sum()

0.30280000000000007

In [73]:
df_keys['in_wiki'].sum()

2642

In [68]:
def get_wiki_key(key):
    try:
        res = httpx.get(
            "https://taginfo.openstreetmap.org/api/4/key/wiki_pages",
            params = {'key': key},
            timeout=15.0
        )
        for r in res.json()['data']:
            if r['lang']=='en':return r
    except TimeoutError:
        return 'timeout'

get_wiki_key('building')

{'lang': 'en',
 'dir': 'ltr',
 'language': 'English',
 'language_en': 'English',
 'title': 'Key:building',
 'description': 'To mark the outline of a building, a man-made structure with a roof, standing more or less permanently in one place.',
 'image': {'title': 'File:Emptyhouse.jpg',
  'width': 1280,
  'height': 800,
  'mime': 'image/jpeg',
  'image_url': 'https://wiki.openstreetmap.org/w/images/6/61/Emptyhouse.jpg',
  'thumb_url_prefix': 'https://wiki.openstreetmap.org/w/images/thumb/6/61/Emptyhouse.jpg/',
  'thumb_url_suffix': 'px-Emptyhouse.jpg'},
 'on_node': True,
 'on_way': False,
 'on_area': True,
 'on_relation': False,
 'tags_implies': ['area=yes'],
 'tags_combination': ['addr', 'name'],
 'tags_linked': ['addr',
  'amenity=hospital',
  'area=yes',
  'building',
  'building:use',
  'building=hospital',
  'building=yes',
  'name'],
 'status': 'approved'}

In [74]:
df_key_with_wiki = df_keys[df_keys['in_wiki']==True]
df_key_with_wiki['wiki'] = df_key_with_wiki['key'].parallel_apply(get_wiki_key)

/var/folders/z5/ks9fxjt91y1dgv38xzprh08m0000gn/T/ipykernel_32039/1581856885.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_key_with_wiki['wiki'] = df_key_with_wiki['key'].parallel_apply(get_wiki_key)


In [76]:
df_key_with_wiki[df_key_with_wiki['wiki'].isnull()].count()

key                         32
count_all                   32
count_all_fraction          32
count_nodes                 32
count_nodes_fraction        32
count_ways                  32
count_ways_fraction         32
count_relations             32
count_relations_fraction    32
values_all                  32
users_all                   32
in_wiki                     32
projects                    32
wiki                         0
dtype: int64

In [78]:
df_key_with_wiki = df_key_with_wiki[ df_key_with_wiki['wiki'].notnull()]
df_key_with_wiki['description'] = df_key_with_wiki['wiki'].apply(lambda x: x['description'])
df_key_with_wiki.to_pickle('keys.pkl')
df_key_with_wiki[['key', 'count_all_fraction', 'description']].to_json(
    "keys.json",
    "records"
)

/var/folders/z5/ks9fxjt91y1dgv38xzprh08m0000gn/T/ipykernel_32039/2727705071.py:4: FutureWarning: Starting with pandas version 3.0 all arguments of to_json except for the argument 'path_or_buf' will be keyword-only.
  df_key_with_wiki[['key', 'count_all_fraction', 'description']].to_json(
